In [20]:
import pandas as pd
from sqlalchemy import create_engine
import requests
from bs4 import BeautifulSoup
import numpy as np

#import seaborn as sns
# Hide warning messages in notebook
import warnings

warnings.filterwarnings('ignore')


In [21]:
dc_df = pd.read_csv("data/dc-wikia-data.csv")
marvel_df =pd.read_csv("data/marvel-wikia-data.csv")
marvel_df.head()

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,Year
0,1678,Spider-Man (Peter Parker),\/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,Brown Hair,Male Characters,NaN,Living Characters,4043.0,Aug-62,1962.0
1,7139,Captain America (Steven Rogers),\/Captain_America_(Steven_Rogers),Public Identity,Good Characters,Blue Eyes,White Hair,Male Characters,NaN,Living Characters,3360.0,Mar-41,1941.0
2,64786,"Wolverine (James \""Logan\"" Howlett)",\/Wolverine_(James_%22Logan%22_Howlett),Public Identity,Neutral Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3061.0,Oct-74,1974.0
3,1868,"Iron Man (Anthony \""Tony\"" Stark)",\/Iron_Man_(Anthony_%22Tony%22_Stark),Public Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2961.0,Mar-63,1963.0
4,2460,Thor (Thor Odinson),\/Thor_(Thor_Odinson),No Dual Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,NaN,Living Characters,2258.0,Nov-50,1950.0


In [22]:
new = marvel_df['name'].str.split('(', n = 1, expand = True)
marvel_df['alias'] = new[0]
marvel_df['alias'] = marvel_df['alias'].str.rstrip()
marvel_df['real_name'] = new[1]
marvel_df['real_name'] = marvel_df['real_name'].str.replace(r')', '')
character_df = marvel_df[['name', 'alias', 'real_name']]
character_df.head()

,name,alias,real_name
0,Spider-Man (Peter Parker),Spider-Man,Peter Parker
1,Captain America (Steven Rogers),Captain America,Steven Rogers
2,"Wolverine (James \""Logan\"" Howlett)",Wolverine,"James \""Logan\"" Howlett"
3,"Iron Man (Anthony \""Tony\"" Stark)",Iron Man,"Anthony \""Tony\"" Stark"
4,Thor (Thor Odinson),Thor,Thor Odinson


In [23]:
dc_df = pd.read_csv("data/dc-wikia-data.csv")
dc_df.head()

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,YEAR
0,1422,Batman (Bruce Wayne),\/wiki\/Batman_(Bruce_Wayne),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3093.0,"1939, May",1939.0
1,23387,Superman (Clark Kent),\/wiki\/Superman_(Clark_Kent),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2496.0,"1986, October",1986.0
2,1458,Green Lantern (Hal Jordan),\/wiki\/Green_Lantern_(Hal_Jordan),Secret Identity,Good Characters,Brown Eyes,Brown Hair,Male Characters,NaN,Living Characters,1565.0,"1959, October",1959.0
3,1659,James Gordon (New Earth),\/wiki\/James_Gordon_(New_Earth),Public Identity,Good Characters,Brown Eyes,White Hair,Male Characters,NaN,Living Characters,1316.0,"1987, February",1987.0
4,1576,Richard Grayson (New Earth),\/wiki\/Richard_Grayson_(New_Earth),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,1237.0,"1940, April",1940.0


In [24]:
marvel_df['universe'] = 'Marvel'
marv_new = marvel_df['name'].str.split('(', n = 1, expand = True)
marvel_df['character'] = marv_new[0]
marvel_df['character'] = marvel_df['character'].str.rstrip()
marvel_df['real_name'] = marv_new[1]
marvel_df['real_name'] = marvel_df['real_name'].str.replace(r')', '')

marvel_df['ALIGN'] = dc_df['ALIGN'].str.replace(r' Characters', '')

marvel_df['SEX'] = dc_df['SEX'].str.replace(r' Characters', '')

marvel_character_df = marvel_df[['name', 'character', 'real_name', 'universe','ALIGN', 'EYE', 'SEX', 'Year']]
marvel_character_df.head()

,name,character,real_name,universe,ALIGN,EYE,SEX,Year
0,Spider-Man (Peter Parker),Spider-Man,Peter Parker,Marvel,Good,Hazel Eyes,Male,1962.0
1,Captain America (Steven Rogers),Captain America,Steven Rogers,Marvel,Good,Blue Eyes,Male,1941.0
2,"Wolverine (James \""Logan\"" Howlett)",Wolverine,"James \""Logan\"" Howlett",Marvel,Good,Blue Eyes,Male,1974.0
3,"Iron Man (Anthony \""Tony\"" Stark)",Iron Man,"Anthony \""Tony\"" Stark",Marvel,Good,Blue Eyes,Male,1963.0
4,Thor (Thor Odinson),Thor,Thor Odinson,Marvel,Good,Blue Eyes,Male,1950.0


In [25]:
dc_df['universe']='DC'
dc_df['Year'] = dc_df['YEAR']
dc_new = dc_df['name'].str.split('(', n = 1, expand = True)
dc_df['character'] = dc_new[0]
dc_df['character'] = dc_df['character'].str.rstrip()
dc_df['real_name'] = dc_new[1]
dc_df['real_name'] = dc_df['real_name'].str.replace(r')', '')

dc_df['ALIGN'] = dc_df['ALIGN'].str.replace(r' Characters', '')

dc_df['SEX'] = dc_df['SEX'].str.replace(r' Characters', '')

dc_character_df = dc_df[['name', 'character', 'real_name', 'universe','ALIGN', 'EYE', 'SEX', 'Year']]
dc_character_df.head()

,name,character,real_name,universe,ALIGN,EYE,SEX,Year
0,Batman (Bruce Wayne),Batman,Bruce Wayne,DC,Good,Blue Eyes,Male,1939.0
1,Superman (Clark Kent),Superman,Clark Kent,DC,Good,Blue Eyes,Male,1986.0
2,Green Lantern (Hal Jordan),Green Lantern,Hal Jordan,DC,Good,Brown Eyes,Male,1959.0
3,James Gordon (New Earth),James Gordon,New Earth,DC,Good,Brown Eyes,Male,1987.0
4,Richard Grayson (New Earth),Richard Grayson,New Earth,DC,Good,Blue Eyes,Male,1940.0


In [26]:
character_table = pd.concat([marvel_character_df, dc_character_df], join="inner")
character_table.head()

,name,character,real_name,universe,ALIGN,EYE,SEX,Year
0,Spider-Man (Peter Parker),Spider-Man,Peter Parker,Marvel,Good,Hazel Eyes,Male,1962.0
1,Captain America (Steven Rogers),Captain America,Steven Rogers,Marvel,Good,Blue Eyes,Male,1941.0
2,"Wolverine (James \""Logan\"" Howlett)",Wolverine,"James \""Logan\"" Howlett",Marvel,Good,Blue Eyes,Male,1974.0
3,"Iron Man (Anthony \""Tony\"" Stark)",Iron Man,"Anthony \""Tony\"" Stark",Marvel,Good,Blue Eyes,Male,1963.0
4,Thor (Thor Odinson),Thor,Thor Odinson,Marvel,Good,Blue Eyes,Male,1950.0


In [27]:
#First appearance of a superhero character
character_table['Year'].min()

1935.0

In [28]:
character_table.isnull().sum()

name             0
character        0
real_name       37
universe         0
ALIGN        10682
EYE          13395
SEX           9730
Year           884
dtype: int64

In [29]:
character_table['EYE'].unique()

array(['Hazel Eyes', 'Blue Eyes', 'Brown Eyes', 'Green Eyes', 'Grey Eyes',
       'Yellow Eyes', 'Gold Eyes', 'Red Eyes', 'Black Eyeballs',
       'Amber Eyes', 'Variable Eyes', nan, 'Black Eyes', 'White Eyes',
       'Orange Eyes', 'Silver Eyes', 'Purple Eyes', 'Pink Eyes',
       'One Eye', 'Violet Eyes', 'Multiple Eyes', 'Magenta Eyes',
       'Yellow Eyeballs', 'No Eyes', 'Compound Eyes',
       'Photocellular Eyes', 'Auburn Hair'], dtype=object)

In [30]:
character_table['SEX'].unique()

array(['Male', 'Female', nan, 'Genderless', 'Transgender'], dtype=object)

In [31]:
print('Male character count is:',character_table['SEX'].value_counts()['Male'])
print('Female character count is:',character_table['SEX'].value_counts()['Female'])

Male character count is: 9566
Female character count is: 3934


In [32]:
#Most characters appeared in these years
most_appearances = character_table['Year'].value_counts()
#most_appearances.head()

In [33]:
most_appearances = most_appearances.to_frame(name=None)
most_appearances.head()

,Year
1993.0,763
1994.0,715
2006.0,684
1992.0,633
2010.0,603


In [34]:
# number of super heros
character_table.shape

(23272, 8)

In [53]:
male_eye = character_table[(character_table['EYE']=='Green Eyes') & (character_table['SEX']=='Male')]

male_eye['ALIGN'].value_counts()

Good                  154
Bad                   147
Neutral                27
Reformed Criminals      1
Name: ALIGN, dtype: int64

In [54]:
female_eye = character_table[(character_table['EYE']=='Green Eyes') & (character_table['SEX']=='Female')]

female_eye['ALIGN'].value_counts()

Good       138
Bad         77
Neutral     32
Name: ALIGN, dtype: int64

In [37]:
print("Characters with variable eyes are:",character_table[character_table.EYE=='Variable Eyes'].name.count())
print("Characters with photocellular Eyes are:",character_table[character_table.EYE=='Photocellular Eyes'].name.count())
print("Characters with compound Eyes are:",character_table[character_table.EYE=='Compound Eyes'].name.count())
print("Characters with no eyes are:",character_table[character_table.EYE=='No Eyes'].name.count())
print("Characters with multiple Eyes are:",character_table[character_table.EYE=='Multiple Eyes'].name.count())
print("Characters with one eyes are:",character_table[character_table.EYE=='One Eye'].name.count())
#print("Characters with Grey eyes are:",character_table[character_table.EYE=='Grey Eyes'].name.count())
#print("Characters with Brown eyes are:",character_table[character_table.EYE=='Brown Eyes'].name.count())

Characters with variable eyes are: 49
Characters with photocellular Eyes are: 48
Characters with compound Eyes are: 1
Characters with no eyes are: 7
Characters with multiple Eyes are: 7
Characters with one eyes are: 21


In [38]:
new = dc_df['name'].str.split('(', n = 1, expand = True)
dc_df['alias'] = new[0]
dc_df['alias'] = dc_df['alias'].str.rstrip()
dc_df['real_name'] = new[1]
dc_df['real_name'] = dc_df['real_name'].str.replace(r')', '')
character2_df = dc_df[['name', 'alias', 'real_name']]
character2_df.head()

,name,alias,real_name
0,Batman (Bruce Wayne),Batman,Bruce Wayne
1,Superman (Clark Kent),Superman,Clark Kent
2,Green Lantern (Hal Jordan),Green Lantern,Hal Jordan
3,James Gordon (New Earth),James Gordon,New Earth
4,Richard Grayson (New Earth),Richard Grayson,New Earth


In [39]:
base_url = 'https://comicvine.gamespot.com/api/'
resource = 'powers'
key = '/?api_key=4313b3a51cfcc09bf7f547f5a726ec5c0be0f790'
url_format = '&sort=name&format=json'

api_call = base_url + resource + key + url_format

user_agent = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

response = requests.get(api_call, headers = user_agent).json()

In [40]:
api_df = pd.DataFrame(response['results'])
api_df.head()

,aliases,api_detail_url,date_added,date_last_updated,description,id,name,site_detail_url
0,None,https://comicvine.gamespot.com/api/power/4035-66/,2008-06-06 11:28:15,2008-06-06 11:28:15,None,66,Adaptive,https://comicvine.gamespot.com/characters/?wik...
1,None,https://comicvine.gamespot.com/api/power/4035-4/,2008-06-06 11:28:15,2008-06-06 11:28:15,<p>Agility is the ability to move nimbly or qu...,4,Agility,https://comicvine.gamespot.com/characters/?wik...
2,None,https://comicvine.gamespot.com/api/power/4035-87/,2008-06-06 11:28:15,2008-06-06 11:28:15,None,87,Animal Control,https://comicvine.gamespot.com/characters/?wik...
3,None,https://comicvine.gamespot.com/api/power/4035-68/,2008-06-06 11:28:15,2008-06-06 11:28:15,None,68,Animation,https://comicvine.gamespot.com/characters/?wik...
4,None,https://comicvine.gamespot.com/api/power/4035-60/,2008-06-06 11:28:15,2008-06-06 11:28:15,None,60,Astral Projection,https://comicvine.gamespot.com/characters/?wik...


In [41]:
key2 = '?api_key=4313b3a51cfcc09bf7f547f5a726ec5c0be0f790'

In [42]:
power =[]
char = []
k = 0
for url in list(api_df['api_detail_url']):
    ul = api_df['api_detail_url'][k]
    ul = ul + key2 + url_format
    rep = requests.get(ul, headers = user_agent).json()
    k+=1
    j=0
    #time.sleep(.2)
    #powers = rep['results']['name']
    for r in rep['results']['characters']:
    #for r in range(1,300):
        try:
            chars = rep['results']['characters'][j]['name']
            char.append(chars)
            powers = rep['results']['name']
            power.append(powers)
            #time.sleep(.2)
            j+=1
        except(IndexError):
            print('done')

In [44]:
powers_df= pd.DataFrame(power, columns = ['power'])
powers_df.head()

,power
0,Adaptive
1,Adaptive
2,Adaptive
3,Adaptive
4,Adaptive


In [45]:
powers_df['power'].unique()

array(['Adaptive', 'Agility', 'Animal Control', 'Animation',
       'Astral Projection', 'Berserker Strength', 'Blast Power',
       'Blood Control', 'Chameleon', 'Chemical Absorbtion',
       'Chemical Secretion', 'Claws', 'Controlled Bone Growth',
       'Cosmic Awareness', 'Danger Sense', 'Darkforce Manipulation',
       'Darkness Manipulation', 'Death Touch', 'Density Control',
       'Dimensional Manipulation', 'Divine Powers', 'Duplication',
       'Earth Manipulation', 'Elasticity', 'Electricity Control',
       'Electronic Disruption', 'Electronic interaction',
       'Emotion Control', 'Empathy', 'Energy Absorption',
       'Energy Based Constructs', 'Energy Manipulation', 'Energy Shield',
       'Energy-Enhanced Strike', 'Enhance Mutation', 'Escape Artist',
       'Feral', 'Fire Control', 'Flame Breath', 'Flight', 'Force Field',
       'Gadgets', 'Genetic Manipulation', 'Gravity control', 'Healing',
       'Heat Generation', 'Heat Vision', 'Hellfire Control',
       'Holograp

In [ ]:
powers_df= pd.DataFrame(power, columns = ['power'])
powers_df.head()

In [ ]:
powers = powers_df['power']value.counts()

In [ ]:
powers_df['character'] = char
#powers_df = powers_df.reset_index()
powers_df 
character_table = pd.merge(character_table, powers_df, how = 'inner', on =['character'])
#character_table['Year'].astype(int)
character_table

In [ ]:
#dc_cols = ["real_name","alias"]
#dc_transformed = dc_df[dc_cols].copy()
#dc_transformed = dc_transformed.rename(columns={"name":"name_x","SEX": "Gender","FIRST APPEARANCE":"First_Appearance" })
                                                          
#dc_transformed.drop_duplicates("name", inplace=True)
#dc_transformed.set_index("name", inplace=True)

#dc_transformed.head()

In [ ]:
#marvel_cols = ["real_name","alias"]
#marvel_transformed = marvel_df[dc_cols].copy()
#marvel_transformed = marvel_transformed.rename(columns={"name":"name_x","SEX": "Gender","FIRST APPEARANCE":"First_Appearance" })
                                                          
#marvel_transformed.drop_duplicates("name", inplace=True)
#marvel_transformed.set_index("name", inplace=True)

#marvel_transformed.head()

In [47]:
rds_connection_string = "root:@127.0.0.1/comic_db"
engine = create_engine(f'mysql+pymysql://{rds_connection_string}')

In [48]:
engine.table_names()

['char_attributes',
 'char_powers',
 'characters',
 'dc_comic',
 'marvel_comic',
 'powers']